# Training the model

In [12]:
%matplotlib inline
import tensorflow as tf
import keras
import collections
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPool2D,GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
import pydot
import random
import keras.backend as K
K.set_image_data_format('channels_last')

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import h5py
import os
from PIL import Image
import PIL.ImageOps
from IPython.display import SVG
import scipy
import numpy as np

In [13]:
train_dataset = h5py.File("./datasets_soilandfire_20200614.h5",'r')
X_train = np.array(train_dataset['x_train'][:])
Y_train = np.array(train_dataset['y_train'][:])
X_test = np.array(train_dataset['x_test'][:])
Y_test = np.array(train_dataset['y_test'][:])
train_dataset.close()
#train_data= create_dataset(X_train,Y_train)

In [14]:
class_names=["fire","soil"]
print(len(X_train))
for i in range(25):
    print(np.squeeze(X_train[i]),np.sum(X_train[i]==1))
    print(class_names[int(Y_train[i])])

38400
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 0. 1. 0.]
 [0. 0. 0. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]] 35
soil
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 1. 0. 0. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 32
soil
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 0. 0. 1. 1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 0. 1.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0.

In [4]:
print("number of traning examples = "+str(X_train.shape[0]))
print("number of test examples = "+str(X_test.shape[0]))
print("X_train shape:" + str(X_train.shape))
print("Y_train shape:" + str(Y_train.shape))
print("X_test shape:" + str(X_test.shape))
print("Y_test shape:" + str(Y_test.shape))
train_data=(X_train,Y_train)
test_data=(X_test,Y_test)

number of traning examples = 38400
number of test examples = 9600
X_train shape:(38400, 10, 10, 1)
Y_train shape:(38400, 1)
X_test shape:(9600, 10, 10, 1)
Y_test shape:(9600, 1)


## Build a model

In [5]:
def Pimodel(input_shape):
    X_input = Input(shape=input_shape)
    X=ZeroPadding2D(padding=(1,1))(X_input)
    X=Conv2D(1,kernel_size=(6,6),strides=(6,6),use_bias=None)(X)
    X=Activation('relu')(X)
    X=MaxPooling2D(pool_size=(2,1))(X)
    X=Flatten()(X)
    Y=Activation('softmax')(X)
    model=Model(inputs=X_input,outputs=Y,name="JSmodel")
    return model

In [6]:
picmodel = Pimodel((10,10,1))

2021-10-05 03:36:58.749458: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-05 03:36:58.749486: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-05 03:36:58.749503: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (8db902df9154): /proc/driver/nvidia/version does not exist
2021-10-05 03:36:58.749834: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-10-05 03:36:58.776025: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2499995000 Hz
2021-10-05 03:36:58.776589: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb314000b20 initialized for platform Host (this does not guar

In [7]:
picmodel.compile(optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [8]:
picmodel.summary()

Model: "JSmodel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 10, 1)]       0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 12, 12, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 2, 2, 1)           36        
_________________________________________________________________
activation (Activation)      (None, 2, 2, 1)           0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 2, 1)           0         
_________________________________________________________________
flatten (Flatten)            (None, 2)                 0         
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0   

In [20]:
history=picmodel.fit(x=X_train,y=Y_train,batch_size=150,epochs=1000,validation_data=(X_test,Y_test))

Epoch 1/1000
20/20 [==============================] - 0s 11ms/step - loss: 0.0361 - accuracy: 0.9900 - val_loss: 0.0391 - val_accuracy: 0.9894
Epoch 2/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0361 - accuracy: 0.9900 - val_loss: 0.0391 - val_accuracy: 0.9893
Epoch 3/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0361 - accuracy: 0.9899 - val_loss: 0.0391 - val_accuracy: 0.9893
Epoch 4/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0361 - accuracy: 0.9900 - val_loss: 0.0390 - val_accuracy: 0.9893
Epoch 5/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0361 - accuracy: 0.9900 - val_loss: 0.0390 - val_accuracy: 0.9893
Epoch 6/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0361 - accuracy: 0.9901 - val_loss: 0.0390 - val_accuracy: 0.9893
Epoch 7/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0360 - accuracy: 0.9900 - val_loss: 0.0390 - val_accuracy: 0.9893
Epoch

20/20 [==============================] - 0s 4ms/step - loss: 0.0355 - accuracy: 0.9901 - val_loss: 0.0385 - val_accuracy: 0.9893
Epoch 59/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0355 - accuracy: 0.9901 - val_loss: 0.0385 - val_accuracy: 0.9893
Epoch 60/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0354 - accuracy: 0.9901 - val_loss: 0.0385 - val_accuracy: 0.9893
Epoch 61/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0354 - accuracy: 0.9901 - val_loss: 0.0385 - val_accuracy: 0.9894
Epoch 62/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0354 - accuracy: 0.9901 - val_loss: 0.0385 - val_accuracy: 0.9894
Epoch 63/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0354 - accuracy: 0.9901 - val_loss: 0.0385 - val_accuracy: 0.9894
Epoch 64/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0354 - accuracy: 0.9901 - val_loss: 0.0385 - val_accuracy: 0.9894
Epoch 65/1000

20/20 [==============================] - 0s 4ms/step - loss: 0.0351 - accuracy: 0.9902 - val_loss: 0.0383 - val_accuracy: 0.9894
Epoch 116/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0351 - accuracy: 0.9902 - val_loss: 0.0383 - val_accuracy: 0.9892
Epoch 117/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0351 - accuracy: 0.9902 - val_loss: 0.0383 - val_accuracy: 0.9892
Epoch 118/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0351 - accuracy: 0.9903 - val_loss: 0.0383 - val_accuracy: 0.9892
Epoch 119/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0351 - accuracy: 0.9903 - val_loss: 0.0383 - val_accuracy: 0.9892
Epoch 120/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0351 - accuracy: 0.9902 - val_loss: 0.0383 - val_accuracy: 0.9894
Epoch 121/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0351 - accuracy: 0.9902 - val_loss: 0.0383 - val_accuracy: 0.9894
Epoch 1

Epoch 172/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0350 - accuracy: 0.9902 - val_loss: 0.0382 - val_accuracy: 0.9893
Epoch 173/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0350 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9893
Epoch 174/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0350 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9893
Epoch 175/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0350 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9894
Epoch 176/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0350 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9893
Epoch 177/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0350 - accuracy: 0.9904 - val_loss: 0.0381 - val_accuracy: 0.9894
Epoch 178/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0350 - accuracy: 0.9904 - val_loss: 0.0381 - val_accuracy:

Epoch 229/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0349 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9894
Epoch 230/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0349 - accuracy: 0.9902 - val_loss: 0.0382 - val_accuracy: 0.9893
Epoch 231/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0349 - accuracy: 0.9902 - val_loss: 0.0382 - val_accuracy: 0.9893
Epoch 232/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0349 - accuracy: 0.9902 - val_loss: 0.0382 - val_accuracy: 0.9894
Epoch 233/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0349 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 234/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0349 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 235/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0349 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy:

...

Epoch 742/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 743/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 744/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 745/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 746/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 747/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 748/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy:

Epoch 799/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 800/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 801/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 802/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 803/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 804/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 805/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy:

Epoch 856/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 857/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9897
Epoch 858/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 859/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 860/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 861/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 862/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy:

Epoch 913/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 914/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 915/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9897
Epoch 916/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 917/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 918/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 919/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy:

Epoch 970/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 971/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 972/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 973/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9903 - val_loss: 0.0382 - val_accuracy: 0.9895
Epoch 974/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 975/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9896
Epoch 976/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.0348 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy:

In [21]:
preds = picmodel.evaluate(x=X_test,y=Y_test)

print('val Loss=' + str(preds[0]))
print(picmodel.name+"Accuracy="+str(preds[1]))

300/300 [==============================] - 0s 656us/step - loss: 0.0382 - accuracy: 0.9896
val Loss=0.038217876106500626
JSmodelAccuracy=0.9895833134651184


In [23]:
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.title('Two-categrorical model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim((0, 1))
plt.text(100,0.4,"accuracy="+str(round(preds[1],3)))
plt.legend(['train accuracy', 'val accuracy'], loc='center right')
plt.show()
#plt.savefig("Two-categrorical_acc.png")

In [24]:
plt.plot(history.history["loss"])
plt.plot(history.history['val_loss'])
plt.title('Two-categrorical model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.text(100,0.4,"loss="+str(round(preds[0],3)))
plt.legend(['train loss', 'val loss'], loc='center right')
#plt.savefig("Two-categrorical_loss.png")
plt.show()

## check the model weight and biases

In [3]:
picmodel=load_model("./model_20200630_soilandfire_rotate.h5")

2021-10-05 03:56:31.777403: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-05 03:56:31.777430: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-05 03:56:31.777449: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (8f56f143247b): /proc/driver/nvidia/version does not exist
2021-10-05 03:56:31.777834: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-10-05 03:56:31.804011: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2499995000 Hz
2021-10-05 03:56:31.804695: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fafdc000b20 initialized for platform Host (this does not guar

In [4]:
kernel_weights=picmodel.layers[2].get_weights()
for i in np.squeeze(kernel_weights):
    print(i)

[-0.06044416 -0.34129035  0.8143321   1.9726045   0.17998758 -4.8121147
 -3.78253     0.9483982   1.8327506   0.78334737 -0.14814235 -0.03438936]
[ 0.08429676  0.20243713 -0.1013227   1.1913661   0.89481926  0.08789831
 -0.1140533   1.3654633   1.0002588   0.04081236 -0.5335867   0.13383302]
[-0.04652618 -0.51260644  0.28135622  0.49143338  1.1595585   0.55704623
  0.87263703  1.0316253   0.7758697  -0.2867816  -0.137779   -0.0208648 ]
[-0.17717262  0.26669312  0.4509626  -0.28149468 -0.6689542  -0.44825947
 -0.05721526 -0.53276056  0.33189765  0.20405546  1.016577    0.07765242]
[ 0.1275447   0.17593543  0.7977436  -0.92301995 -0.78008425  0.9550681
  0.71613693 -0.28919286 -0.52979213  0.12716757  0.28741038  0.1433804 ]
[-0.0081269   0.5272698  -0.07568137 -1.0484308   0.5520728   5.0156684
  4.5047503   1.6711051  -0.61993784  0.12957028  1.1668622  -0.10085674]


In [8]:
def get_layer_output(model,x,index=-1):
    layer = K.function([model.input],[model.layers[index].output])
    return layer([x])[0]

In [9]:
print(picmodel.summary())

Model: "JSmodel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 10, 10, 1)]       0         
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 12, 12, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 1, 1)           72        
_________________________________________________________________
activation_9 (Activation)    (None, 2, 1, 1)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 2)                 0         
_________________________________________________________________
activation_10 (Activation)   (None, 2)                 0         
Total params: 72
Trainable params: 72
Non-trainable params: 0
_______________________________________________________________

In [10]:
kernel_weights=np.squeeze(picmodel.get_weights())
a1=np.squeeze(kernel_weights)[0:6,0:6].flatten()
print(a1)
N=abs(a1.flatten())[np.argmax(abs(a1.flatten()))]
print(N)
a1_nor=a1.flatten()/N
noraml_kernel_weights=(kernel_weights/N)
for j in noraml_kernel_weights:
    print(j)

[-0.06044416 -0.34129035  0.8143321   1.9726045   0.17998758 -4.8121147
  0.08429676  0.20243713 -0.1013227   1.1913661   0.89481926  0.08789831
 -0.04652618 -0.51260644  0.28135622  0.49143338  1.1595585   0.55704623
 -0.17717262  0.26669312  0.4509626  -0.28149468 -0.6689542  -0.44825947
  0.1275447   0.17593543  0.7977436  -0.92301995 -0.78008425  0.9550681
 -0.0081269   0.5272698  -0.07568137 -1.0484308   0.5520728   5.0156684 ]
5.0156684
[-0.01205107 -0.06804484  0.16235764  0.39328846  0.03588506 -0.95941645
 -0.75414276  0.1890871   0.36540505  0.15618005 -0.02953591 -0.00685639]
[ 0.01680669  0.04036095 -0.02020124  0.23752888  0.1784048   0.01752475
 -0.0227394   0.27223954  0.19942681  0.00813697 -0.10638396  0.02668299]
[-0.00927617 -0.10220102  0.05609546  0.09797964  0.23118724  0.11106122
  0.1739822   0.20568052  0.1546892  -0.05717715 -0.02746972 -0.00415992]
[-0.03532383  0.053172    0.08991077 -0.05612306 -0.13337289 -0.08937183
 -0.0114073  -0.10621925  0.06617217  0

## the layer output, you need run the part with raw_datasets,it will run a longtime

In [15]:
path="./datasets/fire/"
imlist=os.listdir(path)
fire_t=[]
Fire_t=[]
fn_t=[]
for elef in imlist:
    input_x = np.array(np.load(r"./datasets/fire/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    fire_t.append(class_names[np.argmax(layer_4)])
    if class_names[np.argmax(layer_4)]=="fire":
        FN_t=[]
        FN_t.append(elef)
        FN_t.append(layer_2.flatten())
        FN_t.append(layer_5.flatten())
        fn_t.append(FN_t)
f1=collections.Counter(fire_t)
print(len(fire_t),collections.Counter(fire_t))

24000 Counter({'fire': 23795, 'soil': 205})


In [16]:
path="./datasets/soil/"
imlist=os.listdir(path)
soil_T=[]
Soil_T=[]
sn_t=[]
for elef in imlist:
    input_x = np.array(np.load(r"./datasets/soil/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    #print(class_names[np.argmax(layer_6)])
    soil_T.append(class_names[np.argmax(layer_4)])
    if class_names[np.argmax(layer_4)]=="soil":
        SN_t=[]
        SN_t.append(elef)
        SN_t.append(layer_2.flatten())
        SN_t.append(layer_4.flatten())
        sn_t.append(SN_t)
        Soil_T.append(elef)
print(len(soil_T))
s1=collections.Counter(soil_T)
print(collections.Counter(soil_T))

24000
Counter({'soil': 23718, 'fire': 282})


def takesecond(elem):
    return elem[0][1]
sn_t.sort(key=takesecond,reverse=True)
print(len(sn_t))
for i in sn_t:
    print(i[0])
    print(i[1]/N)

In [17]:
import seaborn as sns
import pandas as pd
sns.set(style="darkgrid")
f2=dict(f1)
print(f2)
s2=dict(s1)
print(s2)
dict1=pd.Series(f1).reindex(["fire","soil"])
print(dict1.index)
print(dict1.values)
print(sns.barplot(dict1.index,dict1.values))
#print(dict1.plot.bar())
#plt.savefig("fire_rec.png")

{'fire': 23795, 'soil': 205}
{'soil': 23718, 'fire': 282}
Index(['fire', 'soil'], dtype='object')
[23795   205]
AxesSubplot(0.125,0.11;0.775x0.77)


/opt/conda/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [18]:
import seaborn as sns
import pandas as pd
sns.set(style="darkgrid")
s2=dict(s1)
print(s2)
dict2=pd.Series(s2).reindex(["fire","soil"])
print(dict2.index)
print(dict2.values)
print(sns.barplot(dict2.index,dict2.values))
plt.title("recongnition")

{'soil': 23718, 'fire': 282}
Index(['fire', 'soil'], dtype='object')
[  282 23718]
AxesSubplot(0.125,0.11;0.775x0.77)


/opt/conda/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


Text(0.5, 1.0, 'recongnition')